In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
from typing import TypedDict

In [4]:
class Person(TypedDict):
    name: str
    age: int
    

In [5]:
new_person:Person = {"name":"Sahil", "age": 22}
print(new_person)

{'name': 'Sahil', 'age': 22}


In [9]:
from langchain_groq import ChatGroq
api_key=os.getenv("GROQ_API_KEY")
llm=ChatGroq(groq_api_key=api_key,model="gemma2-9b-it")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000270DFE2DDE0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000270DFFF4F10>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [10]:
result=llm.invoke("I had the most incredible dining experience at this restaurant! The food was absolutely delicious - every dish was perfectly prepared and beautifully presented. Our server was attentive, knowledgeable, and made excellent recommendations. The atmosphere was elegant yet comfortable. I can't wait to come back and try more items from their menu. Definitely a new favorite spot!")

In [11]:
result

AIMessage(content="That sounds like a wonderful evening! \n\nI'm so happy to hear you had such a positive experience. It's great to know there are restaurants out there that truly excel in all aspects - from the food and service to the ambiance. \n\nWhat dish was your favorite?  I'm always looking for new restaurant recommendations! \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 74, 'prompt_tokens': 76, 'total_tokens': 150, 'completion_time': 0.134545455, 'prompt_time': 0.005263559, 'queue_time': 0.237977845, 'total_time': 0.139809014}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-5c2a8e22-b621-462d-b29a-f04ac5e327c7-0', usage_metadata={'input_tokens': 76, 'output_tokens': 74, 'total_tokens': 150})

In [ ]:
class review(TypedDict):
    summary: str
    sentiment: str

structured_model=llm.with_structured_output(review)

In [14]:

result=structured_model.invoke("I had the most incredible dining experience at this restaurant! The food was absolutely delicious - every dish was perfectly prepared and beautifully presented. Our server was attentive, knowledgeable, and made excellent recommendations. The atmosphere was elegant yet comfortable. I can't wait to come back and try more items from their menu. Definitely a new favorite spot!")
result

{'sentiment': 'positive',
 'summary': 'The food was delicious, the service was excellent, and the atmosphere was elegant.'}

In [18]:
## No need to give prompt for structured output, it will automatically be inferred from the type hinting

# but some tyepe hinting is required for the output to be structured thats why using Annotatd
from typing import Annotated, TypedDict,Literal,Optional
class review(TypedDict):
    key_themes: Annotated[list[str], "Write down all the key themes discussed in the review in a list"]
    summary: Annotated[str, "A brief summary of the review"]
    sentiment: Annotated[Literal["pos", "neg"], "Return sentiment of the review either negative, positive "]  # literals are used to restrict the output to a set of values
    pros: Annotated[Optional[list[str]], "Write down all the pros inside a list"]  # optional is used to make the field optional
    name: Annotated[Optional[str], "Write the name of the reviewer"]
structured_model=llm.with_structured_output(review)
result=structured_model.invoke("""I recently upgraded to the Samsung Galaxy S24 Ultra, and I must say, it’s an absolute powerhouse! The Snapdragon 8 Gen 3 processor makes everything lightning fast—whether I’m gaming, multitasking, or editing photos. The 5000mAh battery easily lasts a full day even with heavy use, and the 45W fast charging is a lifesaver.

The S-Pen integration is a great touch for note-taking and quick sketches, though I don't use it often. What really blew me away is the 200MP camera—the night mode is stunning, capturing crisp, vibrant images even in low light. Zooming up to 100x actually works well for distant objects, but anything beyond 30x loses quality.

However, the weight and size make it a bit uncomfortable for one-handed use. Also, Samsung’s One UI still comes with bloatware—why do I need five different Samsung apps for things Google already provides? The $1,300 price tag is also a hard pill to swallow.

Pros:
Insanely powerful processor (great for gaming and productivity)
Stunning 200MP camera with incredible zoom capabilities
Long battery life with fast charging
S-Pen support is unique and useful
                                 
Review by Nitish Singh""")       
result

{'key_themes': ['Camera quality',
  'Performance',
  'Battery life',
  'S-Pen',
  'Size and weight'],
 'name': 'Nitish Singh',
 'pros': ['Insanely powerful processor (great for gaming and productivity)',
  'Stunning 200MP camera with incredible zoom capabilities',
  'Long battery life with fast charging',
  'S-Pen support is unique and useful'],
 'sentiment': 'pos',
 'summary': 'The Samsung Galaxy S24 Ultra is a powerful phone with a great camera, long battery life, and useful S-Pen support. However, its size and weight can be uncomfortable, and it comes with bloatware.'}

In [28]:
## Problem in the typedict is that datatype not is always correct as we mention so we use other method 
## pydantic
from pydantic import BaseModel
class student(BaseModel):
    name: str
    age: int
    class_name: str
    roll_no: int

new_studen=student(name="sahil", age=22, class_name="BCA", roll_no=123)
print(new_studen)

name='sahil' age=22 class_name='BCA' roll_no=123


In [ ]:
new_studen=student(name='sahik', age='22', class_name="BCA", roll_no=123)  # have some sence of pydantic like here automoatically convert the age to int
print(new_studen)

name='sahik' age=22 class_name='BCA' roll_no=123


In [30]:
new_studen=student(name=45, age=55, class_name="BCA", roll_no=123)
print(new_studen)

ValidationError: 1 validation error for student
name
  Input should be a valid string [type=string_type, input_value=45, input_type=int]
    For further information visit https://errors.pydantic.dev/2.10/v/string_type

In [ ]:
from pydantic import BaseModel, Field
class student(BaseModel):
    name: str
    age: int
    class_name: str
    roll_no: int 
    cgpa: float = Field(..., gt=0, le=10)  # gt is used to restrict the value to be greater than 0 and le is used to restrict the value to be less than or equal to 10 also add description to the field

new_studen=student(name="sahil", age=22, class_name="BCA", roll_no=123, cgpa=9.5)
print(new_studen)

name='sahil' age=22 class_name='BCA' roll_no=123 cgpa=9.5


In [32]:
new_studen=student(name="sahil", age=22, class_name="BCA", roll_no=123, cgpa=12)
print(new_studen)

ValidationError: 1 validation error for student
cgpa
  Input should be less than or equal to 10 [type=less_than_equal, input_value=12, input_type=int]
    For further information visit https://errors.pydantic.dev/2.10/v/less_than_equal

In [33]:
class review(BaseModel):
    key_themes: list[str] = Field(description="Write down all the key themes discussed in the review in a list")
    summary: str = Field(description="A brief summary of the review")
    sentiment: Literal["pos", "neg"] = Field(description="Return sentiment of the review either negative, positive ")  # literals are used to restrict the output to a set of values    
    pros: Optional[list[str]] = Field(description="Write down all the pros inside a list")  # optional is used to make the field optional
    name: Optional[str] = Field(description="Write the name of the reviewer")
structured_model=llm.with_structured_output(review)
result=structured_model.invoke("""I recently upgraded to the Samsung Galaxy S24 Ultra, and I must say, it’s an absolute powerhouse! The Snapdragon 8 Gen 3 processor makes everything lightning fast—whether I’m gaming, multitasking, or editing photos. The 5000mAh battery easily lasts a full day even with heavy use, and the 45W fast charging is a lifesaver.

The S-Pen integration is a great touch for note-taking and quick sketches, though I don't use it often. What really blew me away is the 200MP camera—the night mode is stunning, capturing crisp, vibrant images even in low light. Zooming up to 100x actually works well for distant objects, but anything beyond 30x loses quality.

However, the weight and size make it a bit uncomfortable for one-handed use. Also, Samsung’s One UI still comes with bloatware—why do I need five different Samsung apps for things Google already provides? The $1,300 price tag is also a hard pill to swallow.

Pros:
Insanely powerful processor (great for gaming and productivity)
Stunning 200MP camera with incredible zoom capabilities
Long battery life with fast charging
S-Pen support is unique and useful
                                 
Review by Nitish Singh""")       
result

review(key_themes=['Performance', 'Camera', 'Battery Life', 'S-Pen', 'Price'], summary='The Samsung Galaxy S24 Ultra is a powerful phone with a stunning camera and long battery life, but its high price and size may be a drawback for some.', sentiment='pos', pros=['Insanely powerful processor (great for gaming and productivity)', 'Stunning 200MP camera with incredible zoom capabilities', 'Long battery life with fast charging', 'S-Pen support is unique and useful'], name='Nitish Singh')